<div style="width:90%;
            margin:10px;
            padding:8px;
            border:2px solid #FF0000;
            border-radius:20px;
            font-weight:bold;
            font-size:2.5em;
            text-align:center;">
The Truck Trailer Problem
</div>


<div style="width:90%;
            margin:9px;
            font-size:1em;">
            
*The Truck Trailer Problem:* ...                
The problem is inspired from the following paper: [FATROP : A Fast Constrained Optimal Control Problem Solver for Robot Trajectory Optimization and Control](https://arxiv.org/pdf/2303.16746) made for [Casadi](https://web.casadi.org/) software and [Fatrop](https://github.com/meco-group/fatrop) solver.                            
We solve the problem using the [JuMP](https://jump.dev/) and [OptimalControl](https://control-toolbox.org/OptimalControl.jl/stable/) packages in Julia then we compare the results.

1. [JuMP Formulation](#jump)
2. [OptimalControl Formulation](#oc)
3. [Comparison](#comp)

</div>

## 1. Solving with JuMP <a id='jump'></a>

In [1]:
using JuMP , Ipopt
import HSL_jll
include("../Problems/JuMP/truck_trailer_JMP.jl")
TruckJMP = truck_JMP();

set_optimizer(TruckJMP,Ipopt.Optimizer)
set_attribute(TruckJMP, "print_level", 5)
set_optimizer_attribute(TruckJMP,"tol",1e-8)
set_optimizer_attribute(TruckJMP,"constr_viol_tol",1e-6)
set_optimizer_attribute(TruckJMP,"max_iter",1000)
set_optimizer_attribute(TruckJMP,"mu_strategy","adaptive")
set_attribute(TruckJMP, "hsllib", HSL_jll.libhsl_path)
set_attribute(TruckJMP, "linear_solver", "ma57")

In [2]:
println("Solving...")
optimize!(TruckJMP)
println()

Solving...

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.14, running with linear solver ma57.

Number of nonzeros in equality constraint Jacobian...:    10612
Number of nonzeros in inequality constraint Jacobian.:     2004
Number of nonzeros in Lagrangian Hessian.............:    55005

Total number of variables............................:     1408
                     variables with only lower bounds:        1
                variables with lower and upper bounds:      804
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1010
Total number of inequali

In [28]:
# Display results
if termination_status(TruckJMP) == MOI.OPTIMAL
    println("  Solution is optimal")
elseif  termination_status(TruckJMP) == MOI.LOCALLY_SOLVED
    println("  (Local) solution found")
elseif termination_status(TruckJMP) == MOI.TIME_LIMIT && has_values(TruckJMP)
    println("  Solution is suboptimal due to a time limit, but a primal solution is available")
else
    error("  The model was not solved correctly.")
end
println("  objective value = ", objective_value(TruckJMP))
println()

ErrorException:   The model was not solved correctly.

## 2. Solving with OptimalControl <a id='oc'></a>

In [12]:
using OptimalControl
include("../Problems/OptimalControl/truck_trailer_OC.jl")
TruckOC = truck_OC();

In [11]:
import HSL_jll
# Solve the problem
sol = OptimalControl.solve(TruckOC,
            linear_solver="ma27",hsllib=HSL_jll.libhsl_path)

Method = (:direct, :adnlp, :ipopt)


MethodError: MethodError: Cannot `convert` an object of type 
  var"#156#160"{ReverseDiff.TrackedArray{ForwardDiff.Dual{ForwardDiff.Tag{ADNLPModels.var"#ψ#268"{CTDirect.var"#36#38"{CTDirect.DOCP}}, Float64}, Float64, 1}, ForwardDiff.Dual{ForwardDiff.Tag{ADNLPModels.var"#ψ#268"{CTDirect.var"#36#38"{CTDirect.DOCP}}, Float64}, Float64, 1}, 1, Vector{ForwardDiff.Dual{ForwardDiff.Tag{ADNLPModels.var"#ψ#268"{CTDirect.var"#36#38"{CTDirect.DOCP}}, Float64}, Float64, 1}}, Vector{ForwardDiff.Dual{ForwardDiff.Tag{ADNLPModels.var"#ψ#268"{CTDirect.var"#36#38"{CTDirect.DOCP}}, Float64}, Float64, 1}}}} to an object of type 
  Union{Real, AbstractVector{<:Real}}

Closest candidates are:
  convert(::Type{T}, !Matched::T) where T
   @ Base Base.jl:64


In [1]:
plot(sol)

UndefVarError: UndefVarError: `plot` not defined